# Mistral-7b : Un guide pour le Fine-Tuning des LLMs
Avec seulement 7 milliards de paramètres, ces modèles sont un choix parfait pour les personnes souhaitant effectuer un fine-tuning des LLMs avec des exigences matérielles limitées.


## Parameter Efficient Fine Tuning : LoRA et Quantification
Les modèles comme Mistral ont rendu possible le fine-tuning gratuitement. Malgré sa petite taille, il nécessite tout de même jusqu'à 30 Go de mémoire GPU. Afin de réduire les exigences en matière de mémoire et les coûts, des techniques comme LoRA et la quantification sont utilisées.

### LoRA
LoRA est une technique d'adaptation de rang faible qui réduit le nombre de paramètres pendant le fine-tuning. LoRA transforme la matrice de paramètres en deux matrices de rang inférieur, dont le produit scalaire approxime la matrice d'origine. Le rang "r" est ajustable, permettant de trouver un équilibre entre la vitesse et la qualité — des rangs inférieurs permettent un entraînement plus rapide, mais au détriment de la qualité.

### Quantification (QLoRA)
Les LLMs utilisent généralement une précision sur 16 bits, ce qui prend beaucoup de mémoire pour stocker des poids de haute précision. La quantification réduit la précision des poids à 4 ou 8 bits, réduisant ainsi considérablement la taille du modèle, au prix d'une légère baisse de qualité. La quantification NF4 convertit les poids à une précision de 4 bits. Ainsi, chaque poids peut prendre jusqu'à 16 valeurs différentes, les normalisant à une distribution centrée sur zéro, ce qui rend les modèles plus efficaces en termes de mémoire tout en ayant un léger impact sur la qualité.


In [1]:
# %pip install -q -U bitsandbytes
# %pip install -q -U git+https://github.com/huggingface/transformers.git
# %pip install -q -U git+https://github.com/huggingface/peft.git
# %pip install -q -U git+https://github.com/huggingface/accelerate.git

In [2]:
# %pip install bitsandbytes
# %pip install transformers
# %pip install peft.git
# %pip install accelerate

In [3]:
import os

In [4]:
import torch
from dotenv import load_dotenv
from huggingface_hub import login

In [5]:
# Définir l'appareil à utiliser (GPU si disponible, sinon CPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [6]:
load_dotenv()
HUGGING_FACE_KEY =  os.environ.get("HuggingFace_API_KEY")
login(token=HUGGING_FACE_KEY)

In [7]:
from datasets import load_dataset

dataset = load_dataset("jpacifico/finetome_french_cook_definitions_v2")

In [8]:
dataset

DatasetDict({
    train: Dataset({
        features: ['system', 'question', 'answer'],
        num_rows: 5000
    })
})

In [9]:
# Définir une fonction pour transformer chaque exemple
def create_input_output(example):
    example['input'] = f"### Instruction: {example['system']} \n ### Question: {example['question']}"
    example['output'] = example['answer']
    return example

In [10]:
# Appliquer la transformation sur chaque exemple du dataset
new_dataset = dataset.map(create_input_output)

# Supprimer les colonnes inutiles ('system', 'question', 'answers')
new_dataset = new_dataset.remove_columns(['system', 'question', 'answer'])

# Pour voir un exemple de ce nouveau dataset avec la colonne 'input'
print(new_dataset['train'][0])  # Afficher le premier exemple

{'input': "### Instruction: Tu es un assistant IA spécialisé dans le langage culinaire français. Une question te sera posée. Tu dois générer une réponse précise et concise. \n ### Question: Quelle est la définition du mot 'Crémer' en cuisine française ?", 'output': "En cuisine française, 'Crémer' signifie ajouter de la crème dans une préparation ou mélanger vigoureusement du sucre et du beurre pommade."}


In [11]:
from transformers import AutoTokenizer

organization = "mistralai/"
model_name = "Mistral-7B-v0.1"
base_model_id = organization + model_name

tokenizer = AutoTokenizer.from_pretrained(
    base_model_id,
    padding_side="left",
    add_eos_token=True,
    add_bos_token=True,
)
tokenizer.pad_token = tokenizer.eos_token

max_length = 500 # Set an appropriate length for the dataset, default 500


In [12]:
# Fonction de formatage pour le prompt
def format_func(input_text, output_text):
    text = f"### Instruction: {input_text}\n ### Question: {output_text}"
    return text


# Fonction de tokenisation adaptée pour les batchs
def tokenize_prompt_max_length(examples):
    # Formatage du texte pour chaque exemple du batch
    formatted_texts = [format_func(input_text, output_text) for input_text, output_text in zip(examples['input'], examples['output'])]
    
    # Tokenisation des textes formatés en batch
    result = tokenizer(
        formatted_texts,
        truncation=True,
        max_length=max_length,
        padding="max_length",
    )
    
    # Créer des labels basés sur les input_ids pour chaque exemple
    result["labels"] = result["input_ids"]
    
    return result


In [13]:
# Appliquer la fonction de tokenisation sur chaque split du dataset
tokenized_dataset = new_dataset.map(tokenize_prompt_max_length, batched=True)


In [14]:
# Pour voir un exemple du dataset tokenisé
print(tokenized_dataset['train'][0])

{'input': "### Instruction: Tu es un assistant IA spécialisé dans le langage culinaire français. Une question te sera posée. Tu dois générer une réponse précise et concise. \n ### Question: Quelle est la définition du mot 'Crémer' en cuisine française ?", 'output': "En cuisine française, 'Crémer' signifie ajouter de la crème dans une préparation ou mélanger vigoureusement du sucre et du beurre pommade.", 'input_ids': [2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,

In [15]:
# Diviser le dataset tokenisé en splits train et évaluation
split_ratio = 0.1  # On prend 10% des données pour l'évaluation
train_test_split = tokenized_dataset['train'].train_test_split(test_size=split_ratio, seed=42)

train_dataset = train_test_split['train']
eval_dataset = train_test_split['test']

In [16]:
print(f"Train dataset size: {len(train_dataset)}")
print(f"Eval dataset size: {len(eval_dataset)}")

Train dataset size: 4500
Eval dataset size: 500


In [17]:
# Vérifier la structure d'un exemple
print(train_dataset[0])

{'input': "### Instruction: Tu es un assistant IA spécialisé dans le langage culinaire français. Une question te sera posée. Tu dois générer une réponse précise et concise. \n ### Question: Que signifie le terme 'Châtrer' dans le contexte du langage culinaire français ?", 'output': 'Châtrer signifie éliminer le boyau central des écrevisses avant leur cuisson.', 'input_ids': [2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2

Maintenant, nous chargeons le modèle pré-entraîné en utilisant la bibliothèque Transformers. Pour cela, nous initialisons le modèle avec une configuration de quantification sur 4 bits via `BitsAndBytesConfig`, ce qui réduit significativement l'utilisation de la mémoire afin de s'assurer que le modèle puisse tenir dans notre session Google Colab.

In [18]:
from transformers import AutoModelForCausalLM, BitsAndBytesConfig

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)


In [19]:
model = AutoModelForCausalLM.from_pretrained(base_model_id, quantization_config=bnb_config)

2024-12-03 14:59:24.537757: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1733234364.558209  146750 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1733234364.564440  146750 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-12-03 14:59:24.585229: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
`low_cpu_mem_usage` was None, now default to True since model is quantized.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [20]:
def generate_response(prompt, max_length=200):
    inputs = tokenizer(prompt, return_tensors="pt").to("cuda")
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_length=max_length,
            num_beams=5,
            no_repeat_ngram_size=2,
            early_stopping=True
        )
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return response


Réponse avec le modèle non fine tuné

In [21]:
prompt = "Que veut dire exactement 'Exprimer' dans le domaine culinaire ?"
# prompt = "### Instruction: Tu es un assistant IA spécialisé dans la cuisine française. \n ### Question: Quelle est la définition du terme 'Exprimer' en cuisine française ?"

response = generate_response(prompt)
print("Réponse:", response)

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


Réponse: Que veut dire exactement 'Exprimer' dans le domaine culinaire ?же как и в других сферах жизни, в кулинарии существует множество терминов, которые могут быть сложными для понимания, особенно для тех, кто только начинает осваивать этот интересный мир. В этой статье мы попытаемся разобраться в том, что означает выражение "exprimer" в контексте приготовления пищи, рассмотрев его в различных аспектах, таких как история, определение, примеры использования и применение в рецептах. Давайте вместе изучим это слово, чтобы лучше понять, как оно влияет


In [22]:
from accelerate import FullyShardedDataParallelPlugin, Accelerator
from torch.distributed.fsdp.fully_sharded_data_parallel import FullOptimStateDictConfig, FullStateDictConfig
from peft import prepare_model_for_kbit_training

fsdp_plugin = FullyShardedDataParallelPlugin(
    state_dict_config=FullStateDictConfig(offload_to_cpu=True, rank0_only=False),
    optim_state_dict_config=FullOptimStateDictConfig(offload_to_cpu=True, rank0_only=False),
)

In [23]:
accelerator = Accelerator(fsdp_plugin=fsdp_plugin)

model.gradient_checkpointing_enable()
model = prepare_model_for_kbit_training(model)

À ce stade, nous configurons LoRA avec "r=16" et "alpha=64" afin de trouver un équilibre entre efficacité et précision lors du fine-tuning. Le paramètre "r" ajuste le rang de la matrice de faible rang : des valeurs plus élevées offrent de meilleures performances, mais au prix d'un fine-tuning plus lent. Le paramètre "alpha" permet de moduler les poids, des valeurs plus élevées donnant davantage d'importance aux activations de LoRA. Ce morceau de code affiche le pourcentage de paramètres entraînables pendant le fine-tuning.


In [24]:
from peft import LoraConfig, get_peft_model

config = LoraConfig(
    r=16,
    lora_alpha=64,
    target_modules=[
        "q_proj",
        "k_proj",
        "v_proj",
        "o_proj",
        "gate_proj",
        "up_proj",
        "down_proj",
        "lm_head",
    ],
    bias="none",
    lora_dropout=0.05,
    task_type="CAUSAL_LM",
)


In [25]:
model = get_peft_model(model, config)

In [26]:
def print_train_params(model):
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
    print(
        f"trainable params: {trainable_params} || all params: {all_param} || trainable%: {100 * trainable_params / all_param}"
    )


In [27]:
print_train_params(model)

trainable params: 42520576 || all params: 3794591744 || trainable%: 1.1205573318192514


In [28]:
if torch.cuda.device_count() > 1: # If more than 1 GPU
    model.is_parallelizable = True
    model.model_parallel = True

model = accelerator.prepare_model(model)

Enfin, nous procédons au fine-tuning du modèle, en définissant `max_steps` à 500 par exemple et un `learning_rate` de 1e-4. 

Nous configurons le `Trainer` pour évaluer le modèle tous les 25 pas, ce qui nous permet de suivre la progression de l'apprentissage. 

En utilisant le paramètre `save_strategy`, nous sauvegardons les poids du modèle à chaque point d'évaluation, c'est-à-dire tous les 25 pas. 

Cette stratégie nous permettra plus tard de sélectionner les poids en fonction des performances d'évaluation. De cette manière, nous pourrons potentiellement faire face au surapprentissage (overfitting) en choisissant des poids obtenus lors d'une phase antérieure de l'entraînement.


In [29]:
import transformers
from datetime import datetime

project = "journal-finetune"
base_model_name = "mistral"
run_name = base_model_name + "-" + project
output_dir = "/mnt/Models/Mistral7B_finetuned_" + run_name

In [30]:
trainer = transformers.Trainer(
    model=model,
    train_dataset = train_dataset,
    eval_dataset  = eval_dataset,
    args=transformers.TrainingArguments(
        output_dir=output_dir,
        warmup_steps=1,
        per_device_train_batch_size=2,
        gradient_accumulation_steps=1,
        gradient_checkpointing=True,
        max_steps=500, # default 500
        learning_rate=1e-4,
        fp16=True,
        fp16_full_eval=True,
        optim="paged_adamw_8bit",
        logging_steps=20,
        logging_dir="./logs",
        save_strategy="steps",
        save_steps=50, #125 default
        evaluation_strategy="steps",
        eval_steps=50, #125 default
        do_eval=True,
    ),
    data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False),
)


/home/azureuser/SourceCode/Lalux/.venv/lib/python3.11/site-packages/transformers/training_args.py:1570: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [31]:
model.config.use_cache = False
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: medhi-famibelle (medhi-famibelle-akabi). Use `wandb login --relogin` to force relogin


Step,Training Loss,Validation Loss
50,0.556700,0.552925
100,0.462700,0.517624
150,0.459600,0.467096
200,0.415000,0.429500
250,0.443500,0.394231
300,0.343100,0.373917
350,0.378400,0.341620
400,0.327000,0.317975
450,0.308100,0.299921
500,0.289100,0.290728


/home/azureuser/SourceCode/Lalux/.venv/lib/python3.11/site-packages/peft/utils/save_and_load.py:230: UserWarning: Setting `save_embedding_layers` to `True` as embedding layers found in `target_modules`.
  warnings.warn("Setting `save_embedding_layers` to `True` as embedding layers found in `target_modules`.")
/home/azureuser/SourceCode/Lalux/.venv/lib/python3.11/site-packages/peft/utils/save_and_load.py:230: UserWarning: Setting `save_embedding_layers` to `True` as embedding layers found in `target_modules`.
  warnings.warn("Setting `save_embedding_layers` to `True` as embedding layers found in `target_modules`.")
/home/azureuser/SourceCode/Lalux/.venv/lib/python3.11/site-packages/peft/utils/save_and_load.py:230: UserWarning: Setting `save_embedding_layers` to `True` as embedding layers found in `target_modules`.
  warnings.warn("Setting `save_embedding_layers` to `True` as embedding layers found in `target_modules`.")
/home/azureuser/SourceCode/Lalux/.venv/lib/python3.11/site-packages

TrainOutput(global_step=500, training_loss=0.4199739284515381, metrics={'train_runtime': 4621.0347, 'train_samples_per_second': 0.216, 'train_steps_per_second': 0.108, 'total_flos': 2.1459542016e+16, 'train_loss': 0.4199739284515381, 'epoch': 0.2222222222222222})

Maintenant que nous allons recharger le modèle dans la session, il n'y aura pas assez de mémoire GPU pour héberger le modèle de base ainsi que le modèle fine-tuné. Par conséquent, nous recommandons de sauvegarder les poids sur votre ordinateur. Les poids peuvent être trouvés à l'adresse : `content/mistral-journal-finetune/checkpoint-xxx/`. Ensuite, redémarrez le kernel et chargez à nouveau les poids.


In [32]:
import torch 
from peft import PeftModel
from transformers import AutoModelForCausalLM, BitsAndBytesConfig, AutoTokenizer

organization = "mistralai/"
model_name = "Mistral-7B-v0.1"
base_model_id = organization + model_name

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)
base_model = AutoModelForCausalLM.from_pretrained(
    base_model_id,
    quantization_config=bnb_config,
    device_map="auto",
    trust_remote_code=True,
)


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [33]:
# ft_model = PeftModel.from_pretrained(base_model, "/content/mistral-journal-finetune/checkpoint-750")
ft_model = PeftModel.from_pretrained(base_model, "/mnt/Models/Mistral7B_finetuned_mistral-journal-finetune/checkpoint-450/")

eval_tokenizer = AutoTokenizer.from_pretrained(base_model_id, add_bos_token=True, trust_remote_code=True)

Enfin, nous pouvons exécuter des inférences sur notre modèle fine-tuné.


In [ ]:
exemple1 = "Quelle est la définition du mot 'Crémer' en cuisine française ?"
exemple2 = "Que veut dire exactement 'Exprimer' dans le domaine culinaire ?"

In [40]:
model_input = eval_tokenizer(exemple1, return_tensors="pt").to("cuda")

ft_model.eval()
with torch.no_grad():
    print(eval_tokenizer.decode(ft_model.generate(**model_input, max_new_tokens=64, repetition_penalty=1.15)[0], skip_special_tokens=True))

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


Quelle est la définition du mot 'Crémer' en cuisine française ?
 ### Crémer : Ajouter de la crème dans une préparation.

Quelle est la définition du mot 'Dessécher' en cuisine française ?
 ### Dessécher : Rendre un aliment moins sec en le plongeant dans un liqu


In [41]:
model_input = eval_tokenizer(exemple2, return_tensors="pt").to("cuda")

ft_model.eval()
with torch.no_grad():
    print(eval_tokenizer.decode(ft_model.generate(**model_input, max_new_tokens=64, repetition_penalty=1.15)[0], skip_special_tokens=True))

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


Que veut dire exactement 'Exprimer' dans le domaine culinaire ?
 ### Exprimer signifie faire sortir du jus des fruits ou des légumes en les pressant. Cela peut également désigner l'action de décrire une recette à haute voix, comme un chef d'un restaurant. En cuisine, exprimer est syn
